In [ ]:
# if working from command line:
!conda install -c conda-forge cartopy
# if working from a shell such as Google Colabatory:
!apt-get -qq install python-cartopy python3-cartopy
!pip uninstall shapely -y
!pip install shapely --no-binary shapely

In [ ]:
'''Import Packages'''
# data wrangling:
import pandas as pd
from math import log, exp
from datetime import datetime as dt, timedelta

# data visualisation:
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import colors
import matplotlib.animation as animation
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader

# define the working directory:
from google.colab import drive
drive.mount('/content/drive' , force_remount=True)
root_dir = '/content/drive/My Drive/'

In [ ]:
'''Import Dataset'''
# Read 'Oil-Gas-Data.csv' from Github, infer the header from the first row of data, parse the column 'Week' as dates:
Dataset = pd.read_csv('https://raw.githubusercontent.com/David-Woroniuk/Medium-Articles/master/O%26G%20Data.csv',
                      header='infer',parse_dates=['Week'])

# Display the first 10 observations:
Dataset.head(10)

In [15]:
'''setup colorbar'''

# normalise colours between the minimum and maximum of the dataset:
norm = mpl.colors.Normalize(vmin=(Dataset['Value'].min()), vmax=(Dataset['Value'].max()))
# set an empty list called colors_in_map:
colors_in_map = []

# set a log base:
logbase = exp(1)
# set the variable 'color_max', which is based on the maximal value of the Dataset:
color_max = (Dataset['Value'].max()+1) 

# for values between 0 and 30:
for i in range(30):
    #     val = (log(i+1)/log(maximal dataset value):
    val = log(i + 1, logbase) / log(color_max, logbase)
    #     append the output to the empty list 'colors_in_map':
    colors_in_map.append((1 - val, val, 0))
# set the colormap equal to the colors listed within our list:
cmap = colors.ListedColormap(colors_in_map)

In [ ]:
'''setup shapefile'''
# we use the natural_earth dataset with a resolution of 1:110, using the map corresponding to the lower 48 states:
shpfilename = shpreader.natural_earth(resolution='110m',
                                      category='cultural',
                                      name='admin_1_states_provinces_lakes_shp')
# Interface to access the contents of a shapefile:
reader = shpreader.Reader(shpfilename)
states_map = reader.records()

# Initialise the facecolor and edgecolor, facecolor is overwritten within 'run' function:
facecolor = 'gray'
edgecolor = 'black'

# plot a figure:
fig = plt.figure()

# use the LambertConformal projection:
ax = fig.add_axes([0, 0, 1, 1], projection=ccrs.LambertConformal())

# set an axis extent, how large the plot should be:
ax.set_extent([-125, -66.5, 20, 50], ccrs.Geodetic())

# remove the background patch and outline patch of the plot:
ax.background_patch.set_visible(False)
ax.outline_patch.set_visible(False)

In [17]:
'''Define init_run Function'''
                                  
# set an emty dictionary called geom_dict:
geom_dict = {}

# define the function init_run:
def init_run():
    for n, states in enumerate(states_map):
        #         Draw all states within our shapefile with a black state boundary, grey color:
        ax.add_geometries(states.geometry, ccrs.PlateCarree(), facecolor=facecolor, edgecolor=edgecolor)
        #         relate the state geometry and the 'name' attribute for each state, save this into geom_dict:
        geom_dict[states.attributes['name']] = states.geometry

In [18]:
'''Define the run function'''

# define the run function:
def run(data):
    # this updates the week based on 'data' value, as our dataset is recorded every 7 days: 
    week = pd.to_datetime('01/08/2010') + timedelta(days=(data*7))
    #     convert into the same format as our dataset:
    week = week.strftime("%d-%m-%Y")

    # get the observation from our Dataset, based on the current week: 
    new_df = Dataset[Dataset['Week'] == week]
    
    for i, row in new_df.iterrows():
        # This loops over countries, gets the value and geometry and adds the new-colored shape:
        geom = geom_dict[row['State']]
        value = row['Value']
        # This sets the colour scheme, and must be consistent with the color bar set above:
        greenamount = (log(float(value) + 1, logbase) / log(color_max, logbase)) 
        # Set the state/shape color:
        facecolor = 1 - greenamount, greenamount, 0
        # add the geometry to the axis using the facecolor and the black outline set outside the function:
        ax.add_geometries(geom, ccrs.PlateCarree(),facecolor=facecolor, edgecolor=edgecolor)
    
    # Update the chart title dependent on the week:
    ax.set_title('US Oil & Gas Drilling Activity ' + str(week))

In [19]:
'''Additional Axes & Colorbar'''
# Add an axis to the figure:
cax = fig.add_axes([0.92, 0.2, 0.02, 0.6])
# add the colorbar which we defined earlier:
cb = mpl.colorbar.ColorbarBase(cax, cmap=cmap, norm=norm,spacing='proportional')
# and provide a label:
cb.set_label('Active Rigs')

In [ ]:
'''Animation'''
# use matplotlib to animate the figure, passing the arguments - 
# the figure, initialisation function, number of frames, interval between frames (ms) and blit (overwriting):
ani = animation.FuncAnimation(fig, run, init_func=init_run, frames=534, interval=15, blit=False)

# use the ffmpeg writer:
Writer = animation.writers['ffmpeg']

# set the animation to have 5 frames per second, with a bitrate of 1800:
writer = Writer(fps=5, metadata=dict(artist='Me'), bitrate=1800)

# save the animation as an mp4 file within root_dir:
ani.save(root_dir +'Drilling Rigs.mp4', writer=writer)

# display the figure:
plt.show()